In [1]:
import torch
import torch.nn.functional as F
import pandas
import torchaudio
import torchaudio.transforms as T
import os
from ..src.utils.datasets import CommonVoiceDataset

from torch.utils.data import Dataset, DataLoader
#from sklearn.model_selection import train_test_split
torchaudio.set_audio_backend("soundfile")

In [11]:
from torchmetrics import Metric
from torch import Tensor
class ExpectedCost(Metric):
    """
    Wrapper on the expected cost as defined in https://github.com/luferrer/expected_cost
    """
    def __init__(self,task = "binary", costs=None, priors=None, sample_weight=None, adjusted=False,num_classes = 2):
        super().__init__()
        self.conf = ConfusionMatrix(task=task,num_classes=num_classes)
        if costs is None:
            
            costs = 1-torch.eye(num_classes)
            
            self.costs = costs
        else:
            self.costs = costs
        self.priors = priors
        self.sample_weight = sample_weight
        self.adjusted=adjusted
        self.num_classes = num_classes
        
            
        self.add_state("bincounts", default=torch.zeros(num_classes), dist_reduce_fx="sum")
        self.add_state("matrix", default=torch.zeros((num_classes,num_classes)), dist_reduce_fx="sum")
        
    def _input_format(self,preds,target):
        return preds,target
    def update(self, preds: Tensor, target: Tensor):
        preds, target = self._input_format(preds, target)
        
        assert preds.shape == target.shape
        self.matrix = self.conf(preds,target)
        
        self.bincounts = torch.bincount(target,minlength=self.num_classes)
        
    def compute(self):
        self.matrix = self.matrix/self.matrix.sum(axis=1, keepdims=True)
        priors = (self.bincounts/self.bincounts.sum()).unsqueeze(0).T
        if self.adjusted:
            # When adjusted is true, normalize the average cost
            # with the cost of a naive system that always makes
            # the min cost decision.
            norm_value = torch.min(torch.dot(priors.T, self.costs))
        else:
            norm_value = 1.0
                
        ave_cost = torch.sum(priors * self.costs * self.matrix)

        return ave_cost / norm_value

### Common Voice (gender)

In [5]:
dataset_train = CommonVoiceDataset(mode="train")


Cleaning.......OK


In [6]:
dataset_valid = CommonVoiceDataset(mode="validation")

Cleaning.......OK


In [3]:
dataset_test = CommonVoiceDataset(mode="test")

Cleaning.......OK


### Dataset test

In [8]:
import IPython.display as ipd


print(f"Length of train set: {len(dataset_train)}")

print(f"Length of valid set: {len(dataset_valid)}")

Length of train set: 364831
Length of valid set: 2566


In [6]:
audio,label = dataset_train.__getitem__(1034)
print(label)
ipd.display(ipd.Audio(audio, rate=16000))

tensor(1)


## Load NMF-based multilabel segmentation model

In [1]:
!python -m pip install numpy==1.23.5

  Using cached numpy-1.23.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.2
    Uninstalling numpy-1.26.2:
      Successfully uninstalled numpy-1.26.2
ERROR: Could not install packages due to an OSError: [Errno 16] Périphérique ou ressource occupé: '.nfs000000000a1f059900000002'



In [9]:
import yaml
from pyannote.audio import Model

w_nmf = torch.load("/lium/raid01_b/tmario/phD/JSALT2023/src/1.explainability/nmf/src/nmf/w_nmf_256_1024_nn_augment.pt")
with open("/lium/raid01_b/tmario/phD/JSALT2023/src/1.explainability/nmf/config_wavlm.yaml","rb") as fh:
    cfg = yaml.safe_load(fh)
model = Model.from_pretrained("/lium/raid01_b/tmario/phD/JSALT2023/src/1.explainability/nmf/3MASNMF/augment/X.Segmentation.Full_WP2-augment-10.0-1.0-0.1-augTrue/models/23_11_14.ckpt",
                              w_nmf=w_nmf,
                              wavlm_cfg=cfg["model"]["wavlm"],
                              spec_kw={"win_length":1024, "hop_length":320,"n_fft":1024})
model.freeze()

Type of NMF matrix: <class 'torch.Tensor'> torch.Size([513, 256])


Using cache found in /lium/home/mlebour/.cache/torch/hub/s3prl_s3prl_main


In [4]:
import yaml
from pyannote.audio import Model

w_nmf = torch.load("/lium/raid01_b/tmario/phD/JSALT2023/src/1.explainability/nmf/src/nmf/w_nmf_256_1024_nn_augment.pt")
with open("/lium/raid01_b/tmario/phD/JSALT2023/src/1.explainability/nmf/config_wavlm.yaml","rb") as fh:
    cfg = yaml.safe_load(fh)
model = Model.from_pretrained("/lium/raid01_b/tmario/phD/JSALT2023/src/1.explainability/nmf/3MASNMF/make_it_sparser_256/X.Segmentation.Full_WP2-make_it_sparser_256-10.0-5.0-0.1-augTrue/models/23_12_12.ckpt",
                              w_nmf=w_nmf,
                              wavlm_cfg=cfg["model"]["wavlm"],
                              spec_kw={"win_length":1024, "hop_length":320,"n_fft":1024})
model.freeze()

Type of NMF matrix: <class 'torch.Tensor'> torch.Size([513, 256])


Using cache found in /lium/home/mlebour/.cache/torch/hub/s3prl_s3prl_main


### Classifier instance

In [93]:
#nmf_order=256
#n_classes = len(dataset_train.get_classes())
#classif_probe=torch.nn.Linear(nmf_order,n_classes,bias=True)

In [8]:
class Probe(torch.nn.Module):
    
    def __init__(self,input_size=256,hidden_size=128,output_size=2):
        super().__init__()
        self.probe = torch.nn.Sequential(torch.nn.Linear(2*input_size,hidden_size),
                            torch.nn.ReLU(),
                            torch.nn.Linear(hidden_size,output_size))
        
    def forward(self,x):
        x_mean = x.mean(dim=1)
        x_std = x.std(dim=1)
        
        y = self.probe(torch.cat([x_mean,x_std],dim=-1))
        
        return y
        
        

In [6]:
class LinProbe(torch.nn.Module):
    
    def __init__(self,input_size=256,output_size=10):
        super().__init__()
        self.probe = torch.nn.Sequential(torch.nn.Linear(2*input_size,output_size))
    
    def forward(self,x):
        x_mean = x.mean(dim=1)
        x_std = x.std(dim=1)
        
        y = self.probe(torch.cat([x_mean,x_std],dim=-1))
        
        return y

In [5]:
class LinProbe(torch.nn.Module):
    
    def __init__(self,input_size=256,output_size=10,agreg = "mean_std"):
        super().__init__()
        self.agreg = agreg
        weight = 1 if agreg in ["mean","std"] else 2
        
        self.probe = torch.nn.Sequential(torch.nn.Linear(weight*input_size,output_size))
    
    def forward(self,x):
        x_mean = None
        x_std = None
        if "mean" in self.agreg:
            x_mean = x.mean(dim=1)
        if "std" in self.agreg:
            x_std = x.std(dim=1)
        if x_mean is not None and x_std is not None:
            input_x = torch.cat([x_mean,x_std],dim=-1)
        elif x_mean is not None:
            input_x = x_mean
        elif x_std is not None:
            input_x = x_std
        
        y = self.probe(input_x)
        
        return y

In [7]:
from tqdm import tqdm

def train_epoch(seg_model,classif_probe,dataloader,loss_fun,optimizer,device="cpu"):
    avg_loss=0
    last_avg_loss=0
    for batch_idx, (data,label) in enumerate(tqdm(dataloader)):    
        optimizer.zero_grad()
        
        with torch.no_grad():
            feat = seg_model.wavlm(data.to(device))
            emb = seg_model.emb_transform(feat)
        
        logits = classif_probe(emb.permute(0,2,1))
        
        loss = loss_fun(logits,label.to(device))
        loss.backward()
        optimizer.step()
        
        avg_loss += loss
        
        if batch_idx%10 == 0:
            avg_loss/=10
            last_avg_loss=avg_loss
            print(f"Train loss: {avg_loss}")
            avg_loss=0
        
    return classif_probe, last_avg_loss

def valid_epoch(seg_model,classif_probe,dataloader,loss_fun,device="cpu"):
    valid_loss=0.0
    for batch_idx, (data, label) in enumerate(tqdm(dataloader,desc="Validation")):        
        
        with torch.no_grad():
            feat = seg_model.wavlm(data.to(device))
            emb = seg_model.emb_transform(feat)
        
        logits = classif_probe(emb.permute(0,2,1))   
        
        valid_loss += loss_fun(logits,label.to(device))
        
    return valid_loss/(batch_idx+1)

In [ ]:
n_epoch = 50

classif_probe = LinProbe(input_size=256,output_size=2)

train_loader= DataLoader(dataset_train, batch_size=64, shuffle=True,num_workers=10)
valid_loader = DataLoader(dataset_valid, batch_size=64, shuffle=False,num_workers=10)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classif_probe.parameters(),lr=0.01)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
classif_probe.to(device)

best_valid = 1e8

train_loss = []
valid_loss = []
for epoch in range(1,n_epoch+1):
    print(f"Epoch {epoch}")
    classif_probe, train_loss_t = train_epoch(seg_model=model,classif_probe=classif_probe,dataloader=train_loader,loss_fun=loss_fn,optimizer=optimizer,device=device)
    valid_loss_t = valid_epoch(seg_model=model,classif_probe=classif_probe,dataloader=valid_loader,loss_fun=loss_fn,device=device)
    print(f"Validation loss = {valid_loss}")
    
    train_loss.append(train_loss_t)
    valid_loss.append(valid_loss_t)
    
    if valid_loss_t < best_valid:
        torch.save(classif_probe.state_dict(), "gender_probe.pt")
        best_valid = valid_loss_t
    

Epoch 1


  0%|          | 1/5701 [00:12<19:51:49, 12.55s/it]

Train loss: 0.06980812549591064


  0%|          | 11/5701 [00:23<3:39:22,  2.31s/it]

Train loss: 0.4540952146053314


...

 89%|████████▊ | 5051/5701 [1:32:27<08:10,  1.33it/s]

Train loss: 0.09782399982213974


 89%|████████▉ | 5061/5701 [1:32:38<07:55,  1.35it/s]

Train loss: 0.11243361234664917


 89%|████████▉ | 5067/5701 [1:32:47<14:40,  1.39s/it]

In [1]:
epochs = range(1,n_epoch+1)
epochs = range(1,20)
train_loss = [i.detach().cpu() for i in train_loss]
valid_loss = [i.detach().cpu() for i in valid_loss]

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(7,5),layout="constrained")
ax.plot(epochs,train_loss,linewidth=2,color="b",label='Train')
ax.plot(epochs,valid_loss,linewidth=2,color="r",label='Valid')
ax.grid()

ax.legend()

ax.set_xlabel("Epoch index")
ax.set_ylabel("Cross Entropy")
ax.set_title("Training curves for gender")

plt.show()

fig.savefig("train_gender_detection.png",dpi=300)

NameError: name 'n_epoch' is not defined

In [6]:
import matplotlib.pyplot as plt

In [7]:
test_loader= DataLoader(dataset_test, batch_size=64, shuffle=False,num_workers=10)

In [19]:
next(iter(test_loader))

[tensor([[-1.2368e-14, -2.2741e-14, -7.3303e-15,  ...,  4.4143e-02,
           7.2552e-02, -6.1471e-03],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -4.5468e-03,
          -6.2040e-03, -1.0184e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -4.1556e-02,
           3.0061e-02,  4.0455e-02],
         ...,
         [-1.1845e-12,  1.1349e-12, -1.3403e-12,  ...,  3.7740e-03,
           5.3200e-03,  5.6067e-03],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  1.5192e-02,
           1.6598e-02,  1.7288e-02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -2.9863e-05,
          -2.8084e-05,  2.0304e-04]]),
 tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0])]

In [8]:
ckpt = torch.load("./probes/sparse_gender_probe_256_mean.pt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
classif_probe = LinProbe(input_size=256,output_size=2,agreg="mean")
classif_probe.load_state_dict(ckpt)
classif_probe.to(device)
model.to(device)

cuda


MASNMF(
  (spec_feat): SpecFeat(
    (spec): Spectrogram()
  )
  (wavlm): WavLM_Feats(
    (feature_extract): UpstreamExpert(
      (model): WavLM(
        (feature_extractor): ConvFeatureExtractionModel(
          (conv_layers): ModuleList(
            (0): Sequential(
              (0): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
              (1): Dropout(p=0.0, inplace=False)
              (2): Sequential(
                (0): TransposeLast()
                (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
                (2): TransposeLast()
              )
              (3): GELU()
            )
            (1): Sequential(
              (0): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
              (1): Dropout(p=0.0, inplace=False)
              (2): Sequential(
                (0): TransposeLast()
                (1): Fp32LayerNorm((512,), eps=1e-05, elementwise_affine=True)
                (2): TransposeLast()
              )
     

In [9]:
from tqdm import tqdm
pred = []
lab = []
for batch_idx, (data,label) in enumerate(tqdm(test_loader)):    

    with torch.no_grad():
        feat = model.wavlm(data.to(device))
        emb = model.emb_transform(feat)
        lab.extend(label.detach().cpu().numpy())
        
        logits = classif_probe(emb.permute(0,2,1))
        #print(logits.shape)
        pred.extend(logits.argmax(dim=-1).detach().cpu().numpy())

        

100%|██████████| 34/34 [00:45<00:00,  1.33s/it]


In [12]:
from torchmetrics import F1Score,Accuracy,ConfusionMatrix,Recall
f1 = F1Score(task="binary")
recall = Recall(task="multiclass",num_classes=2,average="macro")
acc = Accuracy(task='binary')
conf = ConfusionMatrix(task='binary')events
ecp = ExpectedCost(task="binary")

In [13]:
import numpy as np
tpred = torch.from_numpy(np.array(pred))
tlabel = torch.from_numpy(np.array(lab))

In [14]:

from expected_cost import ec
#print("0 Male, 1 Female")
print(f"N Female = {(tlabel==1).sum()} , N Male = {(tlabel==0).sum()}")
print("UAR =",recall(tpred,tlabel))
print("Acc =",acc(tpred,tlabel))
print("F1score Female =",f1(tpred,tlabel))
print("F1score Male =",f1((tpred*-1)+1,(tlabel*-1)+1))
print("Confusion = ",conf(tpred,tlabel))
print("\nCUSTOM EC")
print("Expected cost",ecp(tpred,tlabel))
print("Acc from EC",1-ecp(tpred,tlabel))


N Female = 457 , N Male = 1697
UAR = tensor(0.8960)
Acc = tensor(0.9243)
F1score Female = tensor(0.8260)
F1score Male = tensor(0.9516)
Confusion =  tensor([[1604,   93],
        [  70,  387]])

CUSTOM EC
Expected cost tensor(0.0757)
Acc from EC tensor(0.9243)
